In [22]:
import pandas as pd
import arcpy
import numpy as np
from numpy import mean
from numpy import std
import os
from arcpy import management
# from arcgis.gis import GIS
from time import time
arcpy.env.overwriteOutput = True

workspace = r"D:\\D_IndianaRisk\\data"
arcpy.env.workspace = workspace


In [23]:
#source for Bing building footprints for Indiana (attrubuted with open street maps data)
building_footprints = r"D:\\D_IndianaRisk\\data\\buildings_project.shp"

#set source for streams data
streams = r"D:\\D_IndianaRisk\\data\\streams_project.shp"

#set source for subwatersheds
all_subwatersheds = r"D:\\D_IndianaRisk\\data\\HUCs\\June_6_HUCS.shp"

In [24]:
# arcpy.analysis.Clip("test", "Building_Footprints_Attributed_IN", r"C:\Users\jmatney\Documents\ArcGIS\Projects\Indiana ML Results\Indiana ML Results.gdb\test_Clip", None)

In [4]:
#get distance from residential buildings to streams
#project buildings and streams so they are in the same GCS
# sr = arcpy.SpatialReference(4269)

# arcpy.Project_management(streams, workspace + "streams_project.shp", sr)


# b_copy = arcpy.CopyFeatures_management(building_footprints, workspace + "buildings_copy")


# arcpy.Project_management(b_copy, workspace+"buildings_project.shp", sr)



In [6]:
# #use near tool to get distance to steams
# arcpy.Near_analysis(building_footprints,
#                     streams,
#                     method="GEODESIC")

In [28]:
subwatershed_list = [] 
dist_to_stream_avg_list = []
dist_to_stream_stdev_list =[]


In [ ]:
arcpy.analysis.Clip("test", "Building_Footprints_Attributed_IN", r"C:\Users\jmatney\Documents\ArcGIS\Projects\Indiana ML Results\Indiana ML Results.gdb\test_Clip", None)

In [31]:
#search cursor through each row of county subwatersheds file
with arcpy.da.SearchCursor(all_subwatersheds, ['HUC12', 'AREA', 'PERIMETER']) as cursor:

    for row in cursor:

        subwatershed_number = row[0]
        
        subwatershed_list.append(subwatershed_number)
        
        outputFile = "huc_" + subwatershed_number + "_building.shp"
        
        print(outputFile)
        
        where_clause = "'HUC_12' = '{}'".format(row[0])
        print(where_clause)
        arcpy.SelectLayerByAttribute_management(all_subwatersheds, 'NEW_SELECTION', where_clause)
        arcpy.analysis.Clip(building_footprints, all_subwatersheds, outputFile)

        # get all the distances in to the streams
        dist_to_stream_list = []
        with arcpy.da.SearchCursor(outputFile, ["NEAR_DIST"]) as distCursor: 
            for dist in distCursor:  
                dist_to_stream = dist[0]  
                dist_to_stream_list.append(dist_to_stream)  

        dist_to_stream_avg = mean(dist_to_stream_list)

        dist_to_stream_stdev = std(dist_to_stream_list)

        print(dist_to_stream_avg)
        print(dist_to_stream_stdev)

        dist_to_stream_avg_list.append(dist_to_stream_avg)
        dist_to_stream_stdev_list.append(dist_to_stream_stdev)

huc_051202080201_building.shp
'HUC_12' = '051202080201'
538.9100949635932
417.4282636111914
huc_051201110302_building.shp
'HUC_12' = '051201110302'
538.9100949635932
417.4282636111914
huc_051201060507_building.shp
'HUC_12' = '051201060507'
538.9100949635932
417.4282636111914
huc_071200010303_building.shp
'HUC_12' = '071200010303'
538.9100949635932
417.4282636111914
huc_050800030717_building.shp
'HUC_12' = '050800030717'


KeyboardInterrupt: 

In [32]:
subwatershed_list

['051202080201',
 '051201110302',
 '051201060507',
 '071200010303',
 '050800030717']

In [33]:
dist_to_stream_avg_list

[538.9100949635932, 538.9100949635932, 538.9100949635932, 538.9100949635932]